In [22]:
# target encoding the preprocessed df 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
# loading preprocessed dataset (before encoding)

df = pd.read_csv("pp_no_encoding.csv")
target = 'seller_pass_rate'
print("Loaded dataset:", df.shape)

Loaded dataset: (141447, 46)


In [3]:
# Split the data
X = df.drop(columns=[target])
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add target back to X_train for encoding purposes only
X_train[target] = y_train


In [4]:
# # Create mapping from the original notebook
# seller_country_mapping = X_train[['seller_country', 'seller_pass_rate']].groupby('seller_country')['seller_pass_rate'].mean()
# seller_country_mapping = seller_country_mapping.reset_index()
# seller_country_mapping.columns = ['seller_country', 'seller_country_target_enc']

# # Save the mapping to a CSV file
# seller_country_mapping.to_csv('seller_country_mapping.csv', index=False)


Encoding

In [26]:
# Select categorical columns
categorical_columns = X_train.select_dtypes(include=['object']).columns

# Calculate cardinality for each categorical column
for col in categorical_columns:
    unique_values = X_train[col].nunique()
    print(f"Column: {col}, Cardinality: {unique_values}")


Column: product_type_most_common, Cardinality: 4682
Column: product_gender_target_most_common, Cardinality: 2
Column: product_category_most_common, Cardinality: 6
Column: product_season_most_common, Cardinality: 3
Column: product_condition_most_common, Cardinality: 5
Column: brand_name_most_common, Cardinality: 5137
Column: product_material_most_common, Cardinality: 68
Column: product_color_most_common, Cardinality: 26
Column: seller_badge, Cardinality: 3
Column: warehouse_name_most_common, Cardinality: 6
Column: usually_ships_within_most_common, Cardinality: 5
Column: seller_country, Cardinality: 80


In [27]:
# large cardianlity affects training. too many features 11000 
# using hybrid encoding: target encoding & one hot 


# HYBRID ENCODING

# defining high and low cardinality features based on the suggested strategy
high_cardinality_features = [
    'product_type_most_common', 'brand_name_most_common',
    'product_material_most_common', 'seller_country', 'product_color_most_common'
]

low_cardinality_features = [
    'product_gender_target_most_common', 'product_category_most_common',
    'product_season_most_common', 'product_condition_most_common' , 'seller_badge', 
    'warehouse_name_most_common', 'usually_ships_within_most_common'
]

# target encoding for high cardinality features
for feature in high_cardinality_features:
    # Calculate the mean of the target variable for each category
    category_target_mean = X_train.groupby(feature)['seller_pass_rate'].mean()
    
    # Map the mean to both train and test sets
    X_train[f"{feature}_target_enc"] = X_train[feature].map(category_target_mean)
    X_test[f"{feature}_target_enc"] = X_test[feature].map(category_target_mean)
    
    # Drop the original high-cardinality feature
    X_train.drop(columns=[feature], inplace=True)
    X_test.drop(columns=[feature], inplace=True)

# one-hot encoding for low-cardinality features
X_train = pd.get_dummies(X_train, columns=low_cardinality_features, drop_first=True)
X_test = pd.get_dummies(X_test, columns=low_cardinality_features, drop_first=True)

# align columns of train and test to ensure they match after encoding
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)


In [31]:
# ensuring target variable is dropped after encoding
if target in X_train.columns:
    X_train.drop(columns=[target], inplace=True)

if target in X_test.columns:
    X_test.drop(columns=[target], inplace=True)

In [32]:
print("X_train shape after target encoding:", X_train.shape)
print("X_test shape after target encoding:", X_test.shape)

# before
# print("X_train shape after target encoding:", X_train.shape)
#print("X_test shape after target encoding:", X_test.shape)


X_train shape after target encoding: (113157, 61)
X_test shape after target encoding: (28290, 61)


In [33]:
y_train.shape

(113157,)

In [34]:
y_test.shape

(28290,)

In [39]:
# Load your datasets
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

# Check for missing values in each dataset
print("Missing values in X_train:")
print(X_train.isnull().sum()[X_train.isnull().sum() > 0])

print("\nMissing values in y_train:")
print(y_train.isnull().sum())

print("\nMissing values in X_test:")
print(X_test.isnull().sum()[X_test.isnull().sum() > 0])

print("\nMissing values in y_test:")
print(y_test.isnull().sum())


# missing values due to encoding. likely due to categories in xtest that didnt appear in xtrain so they didnt receive target encoded values

Missing values in X_train:
Series([], dtype: int64)

Missing values in y_train:
seller_pass_rate    0
dtype: int64

Missing values in X_test:
product_type_most_common_target_enc    599
brand_name_most_common_target_enc      522
seller_country_target_enc                5
dtype: int64

Missing values in y_test:
seller_pass_rate    0
dtype: int64


In [40]:
# impute missing values in target-encoded columns in X_test using the mean from X_train
for col in ['product_type_most_common_target_enc', 'brand_name_most_common_target_enc', 'seller_country_target_enc']:
    mean_value = X_train[col].mean()  # Mean of the column in X_train
    X_test[col].fillna(mean_value, inplace=True)

# verifying that missing values are handled
print("\nMissing values in X_test after filling:")
print(X_test.isnull().sum()[X_test.isnull().sum() > 0])



Missing values in X_test after filling:
Series([], dtype: int64)


In [41]:

# target-encoded training set (for class imbalance handling in separate notebooks)
X_train.to_csv("X_train.csv", index=False)
y_train.to_csv("y_train.csv", index=False)

# separate test set (for consistent evaluation across all experiments)
X_test.to_csv("X_test.csv", index=False)
y_test.to_csv("y_test.csv", index=False)
